# 텍스트분석 + 싸이킷런 파이프라인!!
## 텍스트분석 할때 많이 하는 것
- 1) 문서(문장)을 단어나 형태소로 쪼갬
    - 영어는 띄어쓰기 단위로 분리
    - 한글은 형태소를 분석을 통해 형태소 단위로 분리
    - konlpy패키지에 형태소 분석기를 사용해야함

In [1]:
# 명사만 추출하는 함수를 만들어 봅시다

from konlpy.tag import Twitter

def get_noun(text):
    tokenizer = Twitter()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

In [2]:
get_noun("떳다 떳다 비행기 날아라 날아라 높이 높이 날아라 우리 비행기")

['떳', '떳', '비행기', '높이', '높이', '우리', '비행기']

- 2) 문서(단어)를 벡터로 바꿈
    - 문서에 단어가 발생한 빈도 수를 세서 정수 또는 0이나1로 표기된 매트릭스를 만듬
    - 단어의 발생 빈도에 가중치를 부여하여 수치를 변환함 : TF-IDF
    - 뉴럴넷을 활용해(임베딩) 벡터로 바꿈 : word2vec

In [3]:
# 문서 내의 단어빈도를 파악하고 매트릭스로 만듬

from sklearn.feature_extraction.text import CountVectorizer

texts = ["깊은 산속 옹달샘 누가와서 먹나요 토끼?",
         "산토끼 토끼야 어디로 가느냐 깡총깡총 뛰면서",
         "푸른하늘 은하수 하얀 쪽배엔 계수나무 한나무 토끼 한마리",
         "나비야 나비야 이리날아 오너라 호랑나비 흰나비 이리날아 오너라"]

cv = CountVectorizer(tokenizer=get_noun)

In [4]:
cv.fit_transform(texts)

<4x19 sparse matrix of type '<class 'numpy.int64'>'
	with 21 stored elements in Compressed Sparse Row format>

In [5]:
cv.get_feature_names()

['계수나무',
 '나무',
 '나비',
 '나비야',
 '날',
 '누가',
 '마리',
 '산속',
 '산토끼',
 '어디',
 '오너',
 '옹달샘',
 '와',
 '은하수',
 '쪽배',
 '토끼',
 '푸른',
 '하늘',
 '호랑나비']

## 케라스에는 유사한 함수가 있음
- from keras.preprocessing.text import Tokenizer
- 근데 한국어 형태소 분석기를 토크나이저에 적용할 수 없음
- 미리 형태소 분석해놓은 다음에 사용할 수는 있음..그럼 한글은 그냥 사이킷런 CountVectorizer써야겠군

In [9]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()

texts = ["깊은 산속 옹달샘 누가와서 먹나요 토끼?",
         "산토끼 토끼야 어디로 가느냐 깡총깡총 뛰면서",
         "푸른하늘 은하수 하얀 쪽배엔 계수나무 한나무 토끼 한마리",
         "나비야 나비야 이리날아 오너라 호랑나비 흰나비 이리날아 오너라",
         "호랑나비 한마리가 꽃밭에 앉았는데"]

tokenizer.fit_on_texts(texts)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [10]:
tokenizer.word_index

{'가느냐': 14,
 '계수나무': 21,
 '깊은': 6,
 '깡총깡총': 15,
 '꽃밭에': 26,
 '나비야': 2,
 '누가와서': 9,
 '뛰면서': 16,
 '먹나요': 10,
 '산속': 7,
 '산토끼': 11,
 '앉았는데': 27,
 '어디로': 13,
 '오너라': 4,
 '옹달샘': 8,
 '은하수': 18,
 '이리날아': 3,
 '쪽배엔': 20,
 '토끼': 1,
 '토끼야': 12,
 '푸른하늘': 17,
 '하얀': 19,
 '한나무': 22,
 '한마리': 23,
 '한마리가': 25,
 '호랑나비': 5,
 '흰나비': 24}

In [11]:
tokenizer.texts_to_matrix(texts)

array([[0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1.,
        1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
       [0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.]])

In [12]:
# 라벨 만들기 : 1-3은 토끼야이기, 4-5는 나비이야기
import numpy as np
y_data = [0,0,0,1,1]
y_data = np.array(y_data)

## TDM을 머신러닝의 데이터로 사용함
- 현재 데이터는 0-2까지는 토끼이야기, 3-4는 나비이야기임
- 토끼는 0, 나비는 1로 라벨을 정의 하였음

In [10]:
X = tokenizer.texts_to_matrix(texts)
y = y_data

In [11]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier()
text_clf = clf.fit(X[0:4], y[0:4])

In [12]:
text_clf.predict(X[4])

c:\users\ksg\appdata\local\programs\python\python36\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([0])

# pipline은 여러 절차를 한번에 할수 있게 연결해줌
## 이 모든걸 한번에!!!
- 텍스트를 벡터로 바꾸고
- 벡터에 가중치를 적용하고
- 분류기에 적용을 하고

In [181]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

text_clf_svm = Pipeline([('vect', CountVectorizer(tokenizer=get_noun)),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42))])

In [182]:
texts

['깊은 산속 옹달샘 누가와서 먹나요 빌어먹을 토끼',
 '산토끼 토끼야 어디로 가느냐 깡총깡총 빌어먹을 토끼',
 '푸른하늘 은하수 하얀 쪽배엔 계수나무 한나무 토끼 빌어먹을 토끼',
 '나비야 나비야 이리날아 오너라 호랑나비 흰나비 이리날아 오너라',
 '호랑나비 한마리가 꽃밭에 앉았는데']

In [183]:
y

array([0, 0, 0, 1, 1])

In [184]:
text_clf_svm = text_clf_svm.fit(texts, y)

## 새로운 데이터를 테스트해보자

In [185]:
texts_new = ["노래하며 춤추는 나는 아름다운 나비",
            "노래하며 춤추는 나는 빌어먹을 토끼"]

In [186]:
text_clf_svm.predict(texts_new)

array([1, 0])

# 일반 수치형 자료의 경우에도!!!
- 결측값 처리하고
- 데이터 정규화하고
- 파생변수 만들고.... 이런걸 한번에!!!

### 데이터 준비

In [1]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [2]:
fetch_housing_data()

In [3]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [4]:
housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


- 데이터와 라벨을 분리
- 학습데이터셋과 검증 데이터셋 분리

In [15]:
data = housing[housing._get_numeric_data().columns]
X = data.iloc[:,:-1].values
y = data.iloc[:,-1].values

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.3, random_state=1)
print(len(X_train), len(X_test), len(Y_train), len(Y_test))

14448 6192 14448 6192


In [6]:
print(len(X_train), len(X_test), len(Y_train), len(Y_test))

### 결측값 채우고 정규화하는 함수를 파이프라인으로 연결

In [19]:
from sklearn.preprocessing import Imputer
imputer = Imputer(strategy="median")

In [20]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([('imputer', Imputer(strategy="median")),
                         ('std_scaler', StandardScaler()),])

housing_num_tr = num_pipeline.fit_transform(X_train)

In [22]:
housing_num_tr

array([[ 0.64821297, -0.68811806, -1.07853296, ...,  1.53466411,
         1.76494143, -0.50715954],
       [ 0.6531961 , -0.85669979,  0.98848922, ...,  0.20264384,
        -0.11785725, -0.36780162],
       [-0.08929041,  0.52473385,  1.62449604, ..., -0.12284318,
        -0.06286591, -0.72098999],
       ..., 
       [ 0.6531961 , -0.79582306,  1.06799007, ..., -0.57834812,
        -0.54469478, -1.45114921],
       [ 1.2013405 , -0.88947957, -1.47603722, ..., -0.09365549,
        -0.06286591, -0.65684615],
       [-1.31015748,  1.01174774,  0.5114841 , ..., -0.37138083,
        -0.29068717, -0.83089789]])

In [ ]:
# 공식문서에 예시 가져옴

from sklearn import svm
from sklearn.datasets import samples_generator
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.pipeline import Pipeline

# generate some data to play with
X, y = samples_generator.make_classification(n_informative=5, 
                                              n_redundant=0, 
                                              random_state=42)
# ANOVA SVM-C
anova_filter = SelectKBest(f_regression, k=5)
clf = svm.SVC(kernel='linear')
anova_svm = Pipeline([('anova', anova_filter), ('svc', clf)])
anova_svm.set_params(anova__k=10, svc__C=.1).fit(X, y)
                      
Pipeline(memory=None,
         steps=[('anova', SelectKBest()),
                ('svc', SVC())])

prediction = anova_svm.predict(X)
anova_svm.score(X, y)    

# 음 근데 에러나는군

# 연습해보세요!
- http://replet.tistory.com/70?category=667742
- 데이터랑 코드 : https://drive.google.com/file/d/10oRjhYD_-nwXVfl-aP-i7euMZwjE2yi1/view?usp=sharing

In [ ]:
참고자료 1 - https://bbengfort.github.io/tutorials/2016/05/19/text-classification-nltk-sckit-learn.html


참고자료 2 - http://nadbordrozd.github.io/blog/2016/05/20/text-classification-with-word2vec/
